# Amazon Kinesis Data Stream 기초
- Kinesis Data Stream : 스트리밍 데이터를 수집하고 처리 주체에 전달하는 서비스
    - 요청이 아닌 데이터의 처리 (SQS, SNS등은 요청)
    - 단발성 데이터가 아니라 많은 분량으로 지속적으로 전달되는 데이터의 처리
    - 예 : 로그, 이용 데이터, 센서 데이터
- 작은 데이터부터 거의 무한대의 용량까지 처리 가능
- 고가용성을 내부적으로 확보

## Kinesis Data Stream의 주요 구성 요소
- Data Stream : Shard로 구성된 데이터 처리를 위한 스트림
    - Shard : Kinesis의 Data Stream의 처리 단위. 일종의 내부 파이프라인
- 데이터 레코드 : 키네시스를 활용해 보내려 하는 데이터
    - Sequence Number : 샤드 내의 파티션 키 단위로 부여되는 고유 번호
- 프로듀서 : 데이터를 생산해서 키네시스에 전달하는 주체
- 컨슈머 : 데이터를 소비하는 주체로 데이터를 직접 처리하거나 다른 주체로 전달
- 액세스 정책 : 키네시스의 접근을 제어

## Data Stream
- 데이터 처리를 위한 스트림
- 24시간 ~ 1년 단위로 데이터 저장 (SQS는 14일)
    - SQS와 달리 원한다면 이 기간 내에 데이터를 다시 조회/처리 가능
- 샤드 단위로 데이터를 처리
- Capacity Mode
    - On-Demand : 샤드를 수요에 따라 증설하고 사용한 데이터 용량 만큼 비용 지불 (사용량을 예측할 수 없을 때)
    - Provisioned : 샤드 숫자를 지정하고(변경 가능) 샤드의 총 숫자 만큼 비용 지불 (사용량을 예측할 수 있을 때)

### 샤드(Shard)
- 샤드 : Kinesis Data Stream의 처리 단위로 일종의 내부 파이프라인
- 정해진 용량의 데이터를 처리 (하나의 샤드가 처리할 수 있는 데이터 크기)
    - 읽기 : 5트랜잭션/초, 2MB/초
    - 쓰기 : 1000데이터레코드/초, 1MB/초
    - 즉 Data Stream의 처리 가능 용량은 Stream이 보유한 샤드 개수의 합으로 계산 가능
- 데이터 레코드의 Partition Key의 MD5 Hash 값을 기반으로 샤드를 결정(분배 로직)
- 샤드 숫자 조절 방법
    - Reshard : 기존 샤드 분할 혹은 병합
        - 2개로 나누거나(Split) 2개를 합치는(Merge) 것만 가능
    - Update Shard : Background에서 샤드 숫자를 지정한 숫자로 조절(병합/분할)
    - OPEN/CLOSED/EXPIRED 상태로 관리

## Producer
- 총 세가지 방법으로 키네시스에 데이터 전송
    - API : PutRecord, PutRecords로 데이터 레코드 전달
    - Amazon Kinesis Producer Library(KPL) : 키네시스에 데이터 레코드를 전달하기 위한 라이브러리
        - Stream에 데이터를 전달하기 위한 다양한 기능 지원
        - Retry, 데이터 취합 및 Batch 전달, CloudWatch 메트릭을 통한 모니터링
    - Amazon Kinesis Agent : JAVA 기반 어플리케이션

## Consumer
- KCL(Amazon Kinesis Client Library) : Kinesis의 데이터 레코드 읽기를 쉽게 관리하기 위해서 만들어진 라이브러리
    - 자바, 파이썬, 루비, 노드, .NET 지원
    - 데이터 스트림의 변경, 데이터 스트리밍의 로드밸런싱, 리샤드 등 다양한 사왕 및 로직 처리 기능을 미리 구현
- 직접 데이터를 처리하기도 하고 상황에 따라서는 다른 주체(S3, OpenSearch 등)에 전달

## 기타
- KMS를 활용한 서버 사이드 암호화 지원
- IAM 리소스 기반 정책으로 권한 부여 가능
- 클라우드워치에서 다양한 지표 수집
    - Basic : 기본 지표 수집 모드. 스트림 단위의 지표를 1분 단위로 수집
    - Enhanced : 샤드 단위의 지표를 1분 단위로 수집. 추가 비용 존재